In [44]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 3s (92.8 kB/s)
Reading package li

In [45]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [46]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Spark").getOrCreate()
spark

In [47]:
!wget https://raw.githubusercontent.com/Bugg-Bitter/BiG_data_IOT/main/Lab-2/dates.csv
!wget https://raw.githubusercontent.com/Bugg-Bitter/BiG_data_IOT/main/Lab-2/list1.csv
!wget https://raw.githubusercontent.com/Bugg-Bitter/BiG_data_IOT/main/Lab-2/list2.csv

--2022-09-08 15:15:40--  https://raw.githubusercontent.com/Bugg-Bitter/BiG_data_IOT/main/Lab-2/dates.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11490789 (11M) [text/plain]
Saving to: ‘dates.csv.4’

dates.csv.4         100%[===================>]  10.96M  30.4MB/s    in 0.4s    

2022-09-08 15:15:41 (30.4 MB/s) - ‘dates.csv.4’ saved [11490789/11490789]

--2022-09-08 15:15:41--  https://raw.githubusercontent.com/Bugg-Bitter/BiG_data_IOT/main/Lab-2/list1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10232 (10.0K) [text/

In [48]:
d1 = spark.read.csv('dates.csv', header=True, sep=",", inferSchema=True)
d2 = spark.read.csv('list1.csv', header=True, sep=",", inferSchema=True)
d3 = spark.read.csv('list2.csv', header=True, sep=",", inferSchema=True)

d1.show(6)
d2.show(6)
d3.show(6)

+---------+-----+---------------------+-----------+------+-------------+-----------------+-------------------+----+-----+---+----+------+
|User uuid|Likes|Days passed from post|Likes Score|  Type|Numer of Tags|Numer of Comments|        Date Posted|Year|Month|Day|Hour|Minute|
+---------+-----+---------------------+-----------+------+-------------+-----------------+-------------------+----+-----+---+----+------+
|        1|  506|                  657|       0.74| Image|            3|               11|2019-05-12 21:18:39|2019|    5| 12|  21|    18|
|        1|  385|                  657|       0.57| Image|            0|                1|2019-05-12 14:25:55|2019|    5| 12|  14|    25|
|        1|  373|                  666|       0.55|Images|            4|               15|2019-05-03 15:18:37|2019|    5|  3|  15|    18|
|        1|  312|                  667|       0.46|Images|            4|                1|2019-05-02 16:40:38|2019|    5|  2|  16|    40|
|        1|  513|                 

Getting Age Function

In [49]:
d1=d1.withColumnRenamed('Date Posted','date_posted')

In [50]:
from pyspark.sql.functions import lit, to_date, current_date, datediff

def get_age(date):
  age = datediff(current_date(), to_date(date))
  return age/365

In [51]:
d1.select(d1.date_posted, get_age(d1.date_posted).alias("new ages")).show(10)

+-------------------+------------------+
|        date_posted|          new ages|
+-------------------+------------------+
|2019-05-12 21:18:39| 3.328767123287671|
|2019-05-12 14:25:55| 3.328767123287671|
|2019-05-03 15:18:37|3.3534246575342466|
|2019-05-02 16:40:38| 3.356164383561644|
|2019-04-25 16:51:58|3.3753424657534246|
|2019-04-22 20:03:00|3.3835616438356166|
|2019-04-22 15:07:34|3.3835616438356166|
|2019-04-18 14:44:36|3.3945205479452056|
|2019-04-17 15:42:56|3.3972602739726026|
|2019-04-11 15:00:05|3.4136986301369863|
+-------------------+------------------+
only showing top 10 rows



In [52]:
print("The Right Joining")
d3.join(d2, d3.SNo == d2.Rank3, "right").select(d2["Rank3"],d3["Country/Region"],d2["Percentage"]).show(10)

print("The Left Joining")
d3.join(d2, d3.SNo == d2.Rank3, "left").select(d2["Rank5"],d3["Country/Region"],d2["Percentage"]).show(10)

print("The Right Outer Joining")
d3.join(d2, d3.SNo == d2.Rank3, "right_outer").select(d2["Rank5"],d3["Province/State"],d2["Percentage"]).show(10)

print("The Left Outer Joining")
d3.join(d2, d3.SNo == d2.Rank3, "left_outer").select(d2["Rank5"],d3["Last Update"],d2["Percentage"]).show(10)

print("The Full Joining")
d3.join(d2, d3.SNo == d2.Rank3, "full").select(d2["Population"],d3["Last Update"],d2["Percentage"]).show(10)

The Right Joining
+-----+--------------+----------+
|Rank3|Country/Region|Percentage|
+-----+--------------+----------+
|    1|Mainland China|    54.30%|
|    2|Mainland China|    34.45%|
|    3|Mainland China|    75.23%|
|    4|Mainland China|    67.47%|
|    5|Mainland China|    90.87%|
|    6|Mainland China|    76.01%|
|    7|Mainland China|    32.29%|
|    8|Mainland China|    63.85%|
|    9|Mainland China|    84.40%|
|   10|Mainland China|    60.05%|
+-----+--------------+----------+
only showing top 10 rows

The Left Joining
+-----+--------------+----------+
|Rank5|Country/Region|Percentage|
+-----+--------------+----------+
|  116|Mainland China|    54.30%|
|  145|Mainland China|    34.45%|
|   68|Mainland China|    75.23%|
|   83|Mainland China|    67.47%|
|   23|Mainland China|    90.87%|
|   64|Mainland China|    76.01%|
|  150|Mainland China|    32.29%|
|   92|Mainland China|    63.85%|
|   36|Mainland China|    84.40%|
|  103|Mainland China|    60.05%|
+-----+--------------